In [6]:
from utils import MidiProcessor, prepare_data, array_to_midi, concat_all_midi_to_df, get_good_files
from model import build_lstm
import numpy as np
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.utils import np_utils
from keras.models import load_model
import matplotlib.pyplot as plt

In [7]:
input_window_len = 32
pred_steps = 1

In [21]:
artists = {
    'slipknot':{
        'models':{
            'bi_e50':'./best_models/bidirectional_1st_try.h5',
            'lstm_e50': './best_models/lstm_1st_try_50.h5',
            'lstm_e100': './best_models/lstm_2nd_try_100.h5'
        },
        'folder': '../Music'
    },
    'tool':{
        'models':{
            'lstm_e50': './best_models/lstm_1nd_try_50_t.h5'
        },
        'folder': '../Music2'
    },
    'sevendust':{
        'models':{
            'lstm_e50': './best_models/lstm_2nd_try_50_s.h5'
        },
        'folder': '../Music3'
    }
}

In [19]:
files = get_good_files(artists['tool']['folder'])
df, tracks_len_list = concat_all_midi_to_df(files, return_tracks_len_list=True)

../Music2
['Tool_-_The_Grudge_(2).mid', 'Tool_-_The_grudge.mid', 'Tool_-_Schism_(4).mid', 'Tool_-_Sober.mid', 'Tool_-_Forty_Six_&_2.mid', 'Tool_-_The_Patient_(3).mid', 'Tool_-_jimmy_(2).mid', 'Tool_-_Tool_-_Undertow.mid', 'Tool_-_You_Lied.mid', 'Tool_-_46_&_2_(2).mid', 'Tool_-_Reflection.mid', 'Tool_-_Eon_Blue_Apocalypse,_The_Patient.mid', 'Tool_-_Intermission_-_Jimmy.mid', 'Tool_-_Tool_-_Lateralis.mid', 'Tool_-_Part_Of_Me(1).mid', 'Tool_-_Aenima_(2).mid', 'Tool_-_H..mid', 'Tool_-_Ticks_And_Leeches_(2).mid', 'Tool_-_Eulogy_(2)(1).mid', 'Tool_-_Parabol__Parabola.mid', 'Tool_-_Hush.mid', 'Tool_-_Hooker_With_A_Penis.mid', 'Tool_-_Undertow.mid', 'Tool_-_Sweat.mid', 'Tool_-_Prison_Sex.mid', 'Tool_-_Stinkfist.mid', 'Tool_-_Eulogy.mid', 'Tool_-_Intolerance.mid', 'Tool_-_Tool_-_Sober.mid', 'Tool_-_Parabola.mid', 'Tool_-_Lateralus.mid', 'Tool_-_Third_Eye.mid', 'Tool_-_Pushit.mid', 'Tool_-_Flood.mid']
42 drum loops
31 percussion instruments
172314 32-notes


In [31]:
X_train, y_train, X_test, y_test, instruments_list = prepare_data(df, input_window_len=input_window_len, pred_steps=1, overlaps=0, train_test_split=0.1,
                                                                  tracks_len_list=tracks_len_list, max_instruments=10)

In [32]:
model = load_model(artists['tool']['models']['lstm_e50'])

In [33]:
y_pred = (model.predict(X_test) > 0.2).astype("int32")

In [34]:
midiData = array_to_midi(y_pred, instruments_list, 140)

In [35]:
midiData.save('./midi_files/gen_tool_140bpm_e50.mid')